In [11]:
import pandas as pd
import numpy as np

from datetime import datetime, timedelta

import requests
import json

from kiblib.utils.db import DbConn
from kiblib.utils.conf import Config

In [12]:
db_conn = DbConn().create_engine()

webservice_conf = Config().get_config_webservice()
base_url = webservice_conf['base']
username = webservice_conf['user']
password = webservice_conf['pwd']

In [80]:
def get_all_reserves():
    data = []

    url = f"{base_url}/api/v1/holds"
    response = requests.get(url, auth=(username, password))
    data.extend(response.json())

    while 'next' in response.links:
        url = response.links['next']['url']
        response = requests.get(url, auth=(username, password))
        data.extend(response.json())
    return data

def delete_reserve(hold_id, username, password):
    hold_id=str(hold_id)
    url = f"{base_url}/api/v1/holds/{hold_id}"
    response = requests.delete(url, auth=(username, password))
    print(f"{hold_id} : {response.status_code}")

In [84]:
reserves = get_all_reserves()
reserves = pd.DataFrame(reserves)

In [85]:
reserves = reserves[~reserves['status'].isin(['W', 'T'])]
reserves = reserves[reserves['suspended'] == False]
reserves['hold_date'] = pd.to_datetime(reserves['hold_date'])

len(reserves)

2282

In [22]:
query = """
SELECT itemnumber, biblionumber, barcode, homebranch, notforloan, damaged, damaged_on,
itemlost, itemlost_on, withdrawn, withdrawn_on,
itemcallnumber, holdingbranch, location, ccode
FROM koha_prod.items
WHERE location != 'MED0A'
AND notforloan NOT IN (2, 3, 4, 5)
"""
items = pd.read_sql(query, con=db_conn)
items['itemlost_on'] = pd.to_datetime(items['itemlost_on'])
items['withdrawn_on'] = pd.to_datetime(items['withdrawn_on'])
items['damaged_on'] = pd.to_datetime(items['damaged_on'])
len(items)

188396

In [23]:
items2reserves = items[~(
                          (items['itemlost'] == 2) &
                          (items['itemlost_on'] <= datetime.today() - timedelta(days=45) )
                        )]
len(items2reserves)

188346

In [86]:
# cas 1 : réservation d'un document pour retrait à la médiathèque
# les collections du bus sont exclues
reserves_med = reserves[reserves['pickup_library_id'] == 'MED']
items2reserves_med = items2reserves[items2reserves['homebranch'] == 'MED']

biblionumbers = items2reserves_med['biblionumber'].to_list()

reserves_med_ko = reserves_med[~reserves_med['biblio_id'].isin(biblionumbers)]
#reserves_med_ko.to_excel('reserves_med_ko.xlsx', index=False)
reserves_med_ko

,biblio_id,cancellation_date,cancellation_reason,desk_id,expiration_date,hold_date,hold_id,item_id,item_level,item_type,...,non_priority,notes,patron_id,pickup_library_id,priority,status,suspended,suspended_until,timestamp,waiting_date
185,323825,None,None,None,None,2023-03-29,386023,None,False,None,...,False,,60689,MED,1,None,False,None,2023-03-29T09:50:54+02:00,None
196,346959,None,None,None,None,2023-03-31,386334,None,False,None,...,False,None,2549,MED,1,None,False,None,2023-03-31T14:24:57+02:00,None
202,353616,None,None,None,None,2023-04-02,386701,None,False,None,...,False,None,49418,MED,1,None,False,None,2023-07-26T16:40:47+02:00,None
209,224517,None,None,None,None,2023-04-04,386973,None,False,None,...,False,None,64790,MED,1,None,False,None,2023-04-04T23:05:44+02:00,None
232,171998,None,None,None,None,2023-04-08,387796,None,False,None,...,False,None,2151,MED,1,None,False,None,2023-04-08T20:00:08+02:00,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2788,358026,None,None,None,None,2023-07-26,402467,None,False,None,...,False,,59731,MED,1,None,False,None,2023-07-26T11:47:52+02:00,None
2790,358026,None,None,None,None,2023-07-26,402469,None,False,None,...,False,,41040,MED,2,None,False,None,2023-07-26T11:48:10+02:00,None
2791,358039,None,None,None,None,2023-07-26,402470,None,False,None,...,False,,4446,MED,1,None,False,None,2023-07-26T11:55:15+02:00,None
2801,358044,None,None,None,None,2023-07-26,402483,None,False,None,...,False,,67704,MED,1,None,False,None,2023-07-26T14:15:11+02:00,None


In [77]:
r2del = reserves_med_ko[reserves_med_ko['hold_date'] < datetime(2023, 3 , 27)]
r2del_id = r2del['hold_id'].tolist()
len(r2del_id)

35

In [81]:
for hold_id in r2del_id:
    delete_reserve(hold_id, username, password)

378980 : 404
379176 : 404
379785 : 404
380121 : 404
380142 : 404
380362 : 404
380657 : 404
380804 : 204
380887 : 204
381118 : 204
381119 : 204
381120 : 204
381243 : 204
381260 : 204
381335 : 204
381508 : 204
381776 : 204
381814 : 204
381975 : 204
382101 : 204
382335 : 204
382342 : 204
382458 : 204
382640 : 204
383260 : 204
383261 : 204
383674 : 204
384050 : 204
384219 : 204
384281 : 204
384724 : 204
385009 : 204
385113 : 204
385121 : 204
385238 : 404


In [82]:
# cas 2 : réservation d'un document pour retrait au bus
reserves_bus = reserves[reserves['pickup_library_id'] == 'BUS']
items2reserves_bus = items2reserves

biblionumbers = items2reserves_bus['biblionumber'].to_list()

reserves_bus_ko = reserves_bus[~reserves_bus['biblio_id'].isin(biblionumbers)]

In [83]:
reserves_bus_ko

,biblio_id,cancellation_date,cancellation_reason,desk_id,expiration_date,hold_date,hold_id,item_id,item_level,item_type,...,non_priority,notes,patron_id,pickup_library_id,priority,status,suspended,suspended_until,timestamp,waiting_date
152,352456,None,None,None,None,2023-03-04,382358,None,False,None,...,False,,53879,BUS,1,None,False,None,2023-03-04T11:57:47+01:00,None
636,341584,None,None,None,None,2023-05-26,394297,None,False,None,...,False,None,41921,BUS,1,None,False,None,2023-05-26T08:17:20+02:00,None
2078,185879,None,None,None,None,2023-07-15,401237,None,False,None,...,False,,71696,BUS,1,None,False,None,2023-07-15T14:27:11+02:00,None


In [55]:
# on regarde les résas les plus anciennes
reserves_old = reserves[reserves['hold_date'] < datetime(2023, 1 , 26)]
reserves_old = reserves_old[reserves_old['hold_date'] > datetime(2022, 12 , 31)]
reserves_old.to_excel("reserves_old.xlsx")

In [54]:
max_reser = reserves.groupby('biblio_id')['hold_id'].count().reset_index().sort_values(by="hold_id", ascending=False)

,biblio_id,hold_id
855,344335,21
1284,355890,18
1362,356899,18
1163,354485,15
1126,353891,15
...,...,...
532,317933,1
531,317847,1
530,317834,1
529,317796,1
